Dial Detection Predictions

In [ ]:
# Would like a way for the model to know that only one of each dial is possible per image
# Or just increase the required confidence

# Thanks Chat
import cv2
from ultralytics import YOLO

def process_video(input_video_path, output_video_path, model_path):
    # Load the YOLO model
    model = YOLO(model_path)

    # Open the input video
    cap = cv2.VideoCapture(input_video_path)
    
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return
    
    # Get the video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define the codec and create VideoWriter object to save output
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Using XVID codec for better compatibility
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    if not out.isOpened():
        print("Error: Could not open the output video file for writing.")
        cap.release()
        return

    # Process the video frame by frame
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break  # If we can't read a frame, end of video
        
        # Make predictions with YOLO
        results = model(frame, mode="predict", conf=0.5)
        
        # Render predictions on the frame
        frame_with_boxes = results[0].plot()  # This draws boxes on the frame
        
        # Write the processed frame to the output video
        out.write(frame_with_boxes)

        # Optionally, display the frame with predictions (uncomment to view in real-time)
        # cv2.imshow("Frame", frame_with_boxes)
        
        # If you want to quit early, you can add:
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    # Release resources
    cap.release()
    out.release()

    # Close any OpenCV windows
    cv2.destroyAllWindows()
    print(f"Output video saved as: {output_video_path}")

# Example usage:
input_video = "G:/11_17/11_17_24_F1/see_cam_6_2.mp4"  # Replace with your input video path
output_video = 'predictions/output_video.mp4'  # Replace with your desired output path
model_path = "saved_models/m21/weights/best.pt" # Replace with the model path
process_video(input_video, output_video, model_path)

Saving Data scripts

In [ ]:
# Extracts and saves metadata to a matfile
import os
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
from scipy.io import savemat

def extract_metadata(image_path):
    """Extracts metadata including GPS info and timestamp from an image."""
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()  # Get EXIF metadata
        if exif_data is None:
            return None, None, None
        
        metadata = {}
        for tag, value in exif_data.items():
            decoded_tag = TAGS.get(tag, tag)
            metadata[decoded_tag] = value
        
        # Extract GPS information
        gps_info = metadata.get('GPSInfo', None)
        gps_data = {}
        if gps_info:
            for gps_tag in gps_info:
                gps_data[GPSTAGS.get(gps_tag, gps_tag)] = gps_info[gps_tag]
        
        # Extract timestamp
        timestamp = metadata.get('DateTime', None)
        
        # Convert GPS data to latitude and longitude
        if gps_data:
            def convert_to_degrees(value):
                d = value[0] / value[1]
                m = value[2] / value[3]
                s = value[4] / value[5]
                return d + (m / 60.0) + (s / 3600.0)

            lat = convert_to_degrees(gps_data['GPSLatitude']) if 'GPSLatitude' in gps_data else None
            lon = convert_to_degrees(gps_data['GPSLongitude']) if 'GPSLongitude' in gps_data else None
            if gps_data.get('GPSLatitudeRef') == 'S':
                lat = -lat
            if gps_data.get('GPSLongitudeRef') == 'W':
                lon = -lon
        else:
            lat, lon = None, None
        
        return timestamp, lat, lon
    
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None, None, None

def process_folder(folder_path):
    """Processes all images in a folder and extracts metadata."""
    frames = []
    timestamps = []
    latitudes = []
    longitudes = []
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(folder_path, filename)
            timestamp, lat, lon = extract_metadata(image_path)
            frames.append(filename)
            timestamps.append(timestamp)
            latitudes.append(lat)
            longitudes.append(lon)
    
    return frames, timestamps, latitudes, longitudes

if __name__ == "__main__":
    folder_path = input("Enter the folder path containing images: ")
    frames, timestamps, latitudes, longitudes = process_folder(folder_path)
    
    # Save data to a MATLAB file
    matlab_data = {
        'frames': frames,
        'timestamps': timestamps,
        'latitudes': latitudes,
        'longitudes': longitudes
    }
    
    output_file = "metadata.mat"
    savemat(output_file, matlab_data)
    print(f"Data saved to {output_file}")

In [ ]:
# Saving numbers to a new column in a mat file

import scipy.io
import numpy as np

def add_numbers_to_column(mat_file, column_name, numbers, new_column_name=None):
    """
    Adds numbers to a specified column in a MATLAB table (.mat file).
    If the column does not exist, it creates the column with the given numbers.
    Optionally renames the column.

    :param mat_file: Path to the .mat file.
    :param column_name: The name of the column to update or create.
    :param numbers: List of numbers to be added or initialized.
    :param new_column_name: The new name for the column (optional).
    """
    try:
        # Load the .mat file
        mat_data = scipy.io.loadmat(mat_file)

        # Check if the column exists
        if column_name in mat_data:
            # Get the existing column data
            column_data = mat_data[column_name].flatten()
            
            # Ensure the column length matches the numbers length
            if len(numbers) != len(column_data):
                print(f"Length mismatch: Column has {len(column_data)} elements but {len(numbers)} numbers were provided.")
                return

            # Add the numbers to the column
            updated_column = column_data + numbers
        else:
            # Create a new column if it doesn't exist
            print(f"Column '{column_name}' not found. Creating a new column.")
            updated_column = np.array(numbers)

        # Update or create the column in the .mat data
        mat_data[column_name] = updated_column

        # Rename the column if requested
        if new_column_name:
            mat_data[new_column_name] = mat_data.pop(column_name)

        # Save the updated .mat file
        scipy.io.savemat(mat_file, mat_data)
        print(f"Column successfully updated and saved to '{mat_file}'.")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    # Example usage
    mat_file = input("Enter the path to the MATLAB table file (.mat): ")
    column_name = input("Enter the name of the column to update or create: ")
    numbers = input("Enter the list of numbers (comma-separated): ")
    numbers = [float(num.strip()) for num in numbers.split(',')]
    new_column_name = input("Enter the new column name (or press Enter to keep the same): ").strip() or None

    add_numbers_to_column(mat_file, column_name, numbers, new_column_name)